<center><br><br>
    Arkansas Work-Based Learning to Workforce Outcomes <br>
    Applied Data Analytics Training | Spring 2022
    <h1> Presentation Preparation </h1>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Coleridge Initiative</a>
    </span>
    <center>Joshua Edelmann and Benjamin Feder</center>
</center>

***

This notebook provides information on how to prepare research output for disclosure control. It outlines how to prepare different kinds of outputs before submitting an export request and gives an overview of the information needed for disclosure review. _Please read through the entire notebook because it will separately discuss all types of outputs that will be flagged in the disclosure review process._

# AR 2022 Class Export Review Guidelines 

- **Each team will be able to export up to 10 figures/tables**
    
    
- **Every statistic for export must be based on at least 11 individuals and at least 3 employers (when using wage records)**
     - Statistics that are based off of 0-10 individuals must be supressed
     - Statistics that are based off of 0-2 employers must be supressed
    
    
- **All counts will need to be rounded**
    - Counts below 1000 should be rounded to the nearest ten
    - Counts greater than or equal to 1000 should be rounded to the nearest hundred
    > For example, a count of 868 would be rounded to 870 and a count of 1868 would be rounded to 1900

- **All reported wages will need to be rounded to the nearest hundred** 
    
- **All reported averages will need to be rounded to the nearest hundredth** 
   
- **All percentages and proportions need to be rounded**
    - The same rounding rule that is applied to counts must be applied to both the numerator and denominator
    - Percentages must then be rounded to the nearest percent
    - Proportions must be rounded to the nearest hundredth


- **Exact percentiles can not be exported** 
    - Instead, for example, you may calculate a “fuzzy median”, by averaging the true 45th and 55th percentiles
       - If you are calculating the fuzzy percentiles for wage, you will need to round to the nearest hundred after calculating the fuzzy percentile
       - If you are calculating the fuzzy percentile for a number of individuals, you will need to round to the nearest 10 if the count is less than 1000 and to the nearest hundred if the count is greater than or equal to 1000
  
- **Exact Maxima and Minima can not be exported**
    - Suppress maximum and minimum values in general
    - You may replace an exact maximum or minimum with a top-coded value or a fuzzy maximum or minimum value. For example: If the maximum value for earnings is 154,325, it could be top-coded as '100,000+'. And a fuzzy maximum value could be: 
    $$\frac{95th\ percentile\ of\ earnings + 154325}{2}$$
 
 
- **Complementary suppression**
    - If your figures include totals or are dependent on a preceding or subsequent figures, you need to take into account complementary disclosure risks—that is, whether the figure totals or the separate figures when read together, might disclose information about less then 11 individuals in the data in a way that a single, simpler table would not. Team facilitators and export reviewers will work with you by offering guidance on implementing any necessary complementary suppression techniques

##  Supporting Documentation for Exports

For each exported figure, you will need to provide a table with **underlying counts** of individuals and employers (when appropriate) for each statistic depicted in the figure. 

- You will need to include both the rounded and the unrounded counts of individuals

- If percentages or proportions are to be exported, you must report both the rounded and the unrounded counts of individuals for the numerator and denominator. You must also report the counts of employers for both the numerator and the denominator when working with wage records

**Code**
- Please provide the code for every output that needs to be exported and the code generating every table (csv) with underlying counts. It is important for the ADRF staff to have the code to better understand what exactly was done and to be able to replicate results. Understanding how research results are created is important in understanding the research output. Thus, it is important to document every step of the analysis in the Jupyter notebook 

# R Setup

# Goal
In this notebook, we will show you how to implement the above export rules while creating the necessary input files and supporting code to create beautiful visuals that are presentation ready. 

We will cover the following visualizations in this notebook:
- **Bar Plot**: visualizes relationships between numerical and categorical variables
- **Bar Plot with distribution bars**: visualizes relationships between numerical and categorical variables
- **Line Plot**: is commonly used for time series data to show how a variable changes over time
- **Heat Map**: visualizes density

>  Note: Throughout this notebook, we will use colors from the following colorblind-friendly palette:
    "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#999999", "#E69F00",  "#56B4E9", "#F0E442"

In [ ]:
# switching off warnings
options(warn=-1)

#database interaction imports
suppressMessages(library(odbc))

# for data manipulation/visualization
suppressMessages(library(tidyverse))

# scaling data, calculating percentages, overriding default graphing
suppressMessages(library(scales))

# to better view images
# For easier viewing of graphs
# Adjust repr.plot.width and repr.plot.height to change the size of graphs
theme_set(theme_gray(base_size = 24))
options(repr.plot.width = 20, repr.plot.height = 12)
options(warn=0)

# to avoid scientific notation
options(scipen = 999) 

In [ ]:
# Connect to the database
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

# Summary Information
For our first visual, we will create a barplot that will summarise information about our cohort of interest. In this example, we will visualize the number of individuals who completed apprenticeships in the five most common industries by race. As a reminder, our cohort consists of apprenticeship completers in Arkansas between 2015 and 2017. 

Apprenticeship industry and race information for our cohort can be found in the **nb_cohort** table. In the following query, we will read in the **tr_ar_2022.dbo.nb_cohort** table we created in the `02_Creating_a_cohort.ipynb`  notebook.

In [ ]:
query <- "
SELECT *
FROM tr_ar_2022.dbo.nb_cohort nb;
"
cohort <- dbGetQuery(con, query)

head(cohort)

First, we will look at our cohort broken down by race.

In [ ]:
# note: we can use count() since we know there is one row per person
cohort %>%
    count(race)

As you can see, there are fewer than 11 individuals who identify as Asian, Native Hawaiian or Other Pacific Islander, and those that do not wish to answer.  Given that the RAPIDS data is public information, though, the export rules do not apply; thus, we will not need to further aggregate (or eventually round). However, once we link these data to any of the confidential Arkansas microdata (even the crosswalk), we will have to combine these categories to make it possible to export the data.

We suggest aggregating these race variables. Here, for strictly pedagogical purposes, we will create a binary variable for white and non-white.

In [ ]:
cohort <- cohort %>%
    # categorical variable 
    mutate(binary_race = ifelse(race != "White" | is.na(race), "Non-White", "White"))

We also need to find the five most common apprenticeship industries for our cohort.

In [ ]:
top_naics <- cohort %>%
    count(Name) %>%
    arrange(desc(n)) %>%
    # rename to track total individuals
    rename(total_n = n) %>%
    slice(1:5)

top_naics

Now, let's count the number of individuals by the binary race variable within each of these five industries. In cases where we are working with data assets based on any of the protected Arkansas datasets, we would need to make sure we have at least 11 individuals per subgroup. 

In [ ]:
# see counts by Name/binary_race
# keeping total_n in for future sorting in graph
cohort %>%
    inner_join(top_naics, by = 'Name') %>%
    count(binary_race, Name, total_n) %>%
    arrange(binary_race, desc(n))

## Prepare the Data for Export

In general, for each figure/table you create, you should prepare the data for the supplementary table, and then use that data to produce the figure/table. However, since our findings are limited to public data at the moment, we do not need to build a supplemental table containing the counts of individuals (and potentially employers) by data point, and we also do not need to suppress or round any of the values.

Let's still save the underlying table for visualization as a separate data frame.
> Note: If a file for export request only uses public data, be sure to make this clear for the export reviewers in your export documentation.

In [ ]:
Figure_1_data <- cohort %>%
    inner_join(top_naics, by = 'Name') %>%
    count(binary_race, Name, total_n) %>%
    arrange(binary_race, desc(n))

## Visualize the Data: Bar Plot

Now that we have prepared the underlying data for the visualization, we can use the following code to create a bar plot that depicts our cohort broken down by common apprenticeship industries and our binary race variable. When creating this graph, we want to compare our 2 catagories based on the most common apprenticeship industries. Since the comparison variable is **binary_race**, that is what we will input in the `fill` parameter. 

In [ ]:
# colorblind-safe colors in a vector
binary_value_color <- c('Non-White' = "#009E73", 'White' = "#0072B2" ) 
binary_value_color

In [ ]:
Figure_1_data

In [ ]:
Figure_1_data %>%
               # naics on the x-axis, order by most common for entire cohort
    ggplot(aes(x = reorder(Name, total_n), 
               # count on the y-axis
               y = n, 
               # filling bars based on binary_race categories
               fill = binary_race)) +
    geom_bar(stat = "identity", position = 'dodge') +
    scale_fill_manual("Binary Race Indicator", values = binary_value_color) + # label legend and assign color palette 
    labs(
        x = 'Industry', # labelling x axis
        y = 'Number of Individuals', # labelling y axis
        # Add a title that conveys the main takeaway of the graph
        title = 'Race Distribution Amongst Most Common Industries Differs for Apprenticeship Completers', # \n splits the title into two lines to avoid getting cut off if needed
        caption = 'Source: RAPIDS Data' # cite the source of your data
    ) 

It is hard to read the industry descriptions due to overlap. Let's shorten them.

In [ ]:
# shorten industry descriptions
# using TRUE at the end because it is the last category
# note that "Commerical and Institutional Building Construction" has an extra space, can trim all whitespace as well
Figure_1_data <- Figure_1_data %>%
    mutate(
        Name = case_when(
            Name == 'Painting and Wall Covering Contractors' ~ 'Painting Contractor',
            Name == 'Commercial and Institutional Building Construction ' ~ 'Commercial Construction',
            Name == 'Electrical Contractors and Other Wiring Installation Contractors' ~ 'Electrical Contractor',
            Name == 'Finish Carpentry Contractors' ~ 'Carpentry Contractor',
            TRUE ~ 'Appliance Contractor'
        )
    )

In [ ]:
Figure_1 <- Figure_1_data %>%
               # naics on the x-axis, order by most common for entire cohort
    ggplot(aes(x = reorder(Name, total_n), 
               # count on the y-axis
               y = n, 
               # filling bars based on binary_race categories
               fill = binary_race)) +
    geom_bar(stat = "identity", position = 'dodge') +
    scale_fill_manual("Binary Race Indicator", values = binary_value_color) + # label legend and assign color palette 
    labs(
        x = 'Industry', # labelling x axis
        y = 'Number of Individuals', # labelling y axis
        # Add a title that conveys the main takeaway of the graph
        title = 'Race Distribution Amongst Most Common Industries Differs for Apprenticeship Completers', # \n splits the title into two lines to avoid getting cut off if needed
        caption = 'Source: RAPIDS Data' # cite the source of your data
    ) 

# see plot
Figure_1

### Adjusting Font Sizes
In order to make the plot presentation ready, we advise using readable font sizes, as the image will be added to either a presentation or report. Additionally, since the NAICS descriptions still long, it may make sense to rotate the labels to improve the readability.

In [ ]:
Figure_1 <- Figure_1 + 
    theme(
        legend.text = element_text(size = 24), # legend text font size
        legend.title = element_text(size = 24), # legend title font size
        axis.text.x = element_text(size = 24), # x axis label font size
        axis.title.x = element_text(size = 24), # x axis title font size
        axis.text.y = element_text(size = 24), # y axis label font size
        axis.title.y = element_text(size = 24) # y axis title font size
    ) + 
    scale_x_discrete(
        guide = guide_axis(angle = 45) # rotate x-axis labels
    )

# Display the graph that we just created
Figure_1

# Fuzzy Wage Quantiles

You might not want a figure that only shows number of individuals broken down by apprenticeship industry and a binary race variable, and want to convey information about the distribution of primary employment wages by race relative to the quarter after apprenticeship completion. 

To do so, we will eventually create a bar graph where each bar will represent a fuzzy median, and we will include distribution bars on each bar that represent the fuzzy 25th and 75th quantiles of the wages for each grouping.

## Steps for Export

We will adhere to the following steps in preparing this visualization for export:

1. Create fuzzy percentiles
    - Fuzzy 25th percentile: Calculate the 20th and 30th percentiles and take the average 
    - Fuzzy median : Calculate the 45th and 55th percentiles and take the average
    - Fuzzy 75th percentile: Calculate the 70th and 80th percentiles and take the average

To get the average, we add the percentiles together and divide by 2. For example:
$$fuzzy\  25th\ = \frac{20th + 30th}{2}$$

2. Redact values 
    - Values with employer counts below 3 and/or individual counts below 11 must be removed

3. Round values
    - Counts below 1000 should be rounded to the nearest ten
    - Counts above or equal to 1000 should be rounded to the nearest hundred
    - Wages must be rounded to the nearest hundred

## Prepare the Data for Export

Recall that the data frame **cohort** only contains information from RAPIDS - in order to answer this question, we will need to bring in information from the wage records. 

In [ ]:
qry <- "
SELECT C.race,
F.Quarter_ID - P.Apprenticeship_End_Quarter_ID AS Quarters_Relative_to_Completion,
P.Person_ID,
F.Primary_Employer_Wages ,
PE.Federal_EIN,
C.apprnumber --pulling in for later
FROM 
tr_ar_2022.dbo.nb_cohort C --COHORT
JOIN tr_ar_2022.dbo.AR_MDIM_Person P ON (P.Apprentice_Number=C.apprnumber) --PERSON
JOIN tr_ar_2022.dbo.AR_FACT_Quarterly_Observation F --QUARTERLY OBSERVATION FACT
    ON (P.Person_ID=F.Person_ID) 
    AND (F.Quarter_ID BETWEEN (P.Apprenticeship_End_Quarter_ID + 1) AND (P.Apprenticeship_End_Quarter_ID+4))  --QTRS POST COMPLETION
JOIN tr_ar_2022.dbo.AR_RDIM_NAICS_National_Industry NNI ON (P.Apprenticeship_NAICS_National_Industry_ID=NNI.NAICS_National_Industry_ID) --APPRENTICESHIP INDUSTRY
JOIN tr_ar_2022.dbo.AR_MDIM_Employer PE ON (PE.Employer_ID=F.Primary_Employer_ID)  --PRIMARY EMPLOYER
"

# create binary race variable
cohort_wages <- dbGetQuery(con, qry) %>%
    mutate(binary_race = ifelse(race != "White" | is.na(race), "Non-White", "White"))
    
head(cohort_wages)

We can apply our disclosure rules to our table by including the rules in the `ifelse()` statement below. We can also apply our rounding rules.

> Note: We are replacing all values that do not satisfy our disclosure rules with `NA`.

In [ ]:
# round and apply disclosure rules
# first calculating fuzzy medians and finding number of unique individuals and employers
Figure_2_data <-  cohort_wages %>%
    group_by(Quarters_Relative_to_Completion,binary_race) %>%
    summarise(
        individuals = n_distinct(Person_ID),
        employers = n_distinct(Federal_EIN),
        # fuzzy 25th percentile
        fuzzy_25 = (quantile(Primary_Employer_Wages, .20) + quantile(Primary_Employer_Wages, .30))/2,
        # fuzzy median (50th percentile)
        fuzzy_median = (quantile(Primary_Employer_Wages, .45) + quantile(Primary_Employer_Wages, .55))/2,
        # fuzzy 75th percentile
        fuzzy_75 = (quantile(Primary_Employer_Wages, .70) + quantile(Primary_Employer_Wages, .80))/2
    ) %>%
    ungroup() %>%
    # if the subgroup satisfies disclosure rules (using unrounded values), round to nearest 100
    # otherwise redact
    mutate(
        fuzzy_25_rounded = ifelse(individuals < 11 | employers < 3, NA, round(fuzzy_25, -2)),
        fuzzy_median_rounded = ifelse(individuals < 11 | employers < 3, NA, round(fuzzy_median, -2)),
        fuzzy_75_rounded = ifelse(individuals < 11 | employers < 3, NA, round(fuzzy_75, -2))
    )

Figure_2_data

This data frame now has all of the necessary underlying information for export review, and because it has been built based in part on the protected microdata, we will need to include it in the export submission. Let's save this data frame as a csv.

This table will not be exported, it will be used by the export team to make sure the figure passes the disclosure requirments. As in this example, all supporting tables should be generated programmatically (using only code) and the associated figure should be generated only using data that correspond to that table.

> **Note: In order to save the data in this way, you will need a folder called "Data" in the same folder that contains your code/notebooks.**

In [ ]:
write_csv(Figure_2_data, "Data/Figure_2_data.csv")

Although none of the rows needed to be primarily suppressed, some of these rows may need to be suppressed when evaluated in combination with other files. Your team lead and export reviewer will help with that.

## Visualize the Data: Bar Plot with Distribution Bars

Since that we have prepared our data and calculated fuzzy percentiles, we can now use the following code to create a bar plot that depicts the (fuzzy) median wage for each subcategory. Similar to what we did above, we want to compare the median wage for our binary race variable relative to the quarter after apprenticeship completion. Since the comparison variable is `binary_race`, that is what we will input in the `fill` parameter. 

Furthermore, we use the `geom_errorbar()` function to add distribution bars that reflect the (fuzzy) 25th and 75th percentiles for wages. 

> Note: Just because you use the `geom_errorbar()` function does not mean you have to depict standard errors. You may use it to depict other distribution information such as the 25th and 75th percentiles. 

In [ ]:
Figure_2 <- Figure_2_data %>%
    ggplot(aes(x = Quarters_Relative_to_Completion, 
               y = fuzzy_median, 
               fill = binary_race)) +
    geom_bar(stat="identity", position='dodge') +
    # adding distribution bars
    geom_errorbar(aes(ymin = fuzzy_25, 
                      ymax = fuzzy_75),
                width = .2,                 
                position = position_dodge(.9)) +
    scale_fill_manual("Binary Race Variable", values = binary_value_color) +
    labs(
        x = 'Quarter Relative to Completion', # labelling x axis
        y = 'Fuzzy Median Percentile Values', # labelling y axis
        # Add a title that conveys the main takeaway of the graph
        title = 'REDACTED', # The \n splits the title into two lines 
        caption = 'Source: RAPIDS and UI Wage data' # cite the source of your data
        ) 
Figure_2

Adjust the font size using the code below:

In [ ]:
Figure_2 <- Figure_2 +
   theme(
        legend.text = element_text(size = 24), # legend text font size
        legend.title = element_text(size = 24), # legend title font size
        axis.text.x = element_text(size = 24), # x axis label font size
        axis.title.x = element_text(size = 24), # x axis title font size
        axis.text.y = element_text(size = 24), # y axis label font size
        axis.title.y = element_text(size = 24) # y axis title font size
    )

# Display the graph that we just created
Figure_2

# Percent Employed over Time

From these first two visuals, we can build one tracking the percent employed by quarter relative to completion, broken down by the binary race variable. The graph will depict the percent of individuals from our cohort employed relative to completion in a line graph. 

## Steps for Export

We will adhere to the following steps in preparing this visualization for export:

1. Redact values 
    - Values with employer counts below 3 and/or individual counts below 11 must be removed

2. Round values

    - Counts below 1000 should be rounded to the nearest ten
    - Counts above or equal to 1000 should be rounded to the nearest hundred
    - Wages must be rounded to the nearest hundred
   
3. Find and round percent
    - Must be derived from the rounded counts - round to the nearest percent

## Prepare the Data for Export

We already have the number of individuals employed by quarter relative to apprenticeship completion for our subgroup of interest (**Figure_2_data**), and we need to combine that with the total amount of individuals by the binary race indicator from our original cohort because we want to find the proportion of our cohort of apprenticeship completers that have a wage.

In [ ]:
# total by binary_race
total_race <- cohort %>%
    count(binary_race) %>%
    # rename n to n_total
    rename(n_total = n)

total_race

In [ ]:
# join figure_2_data with total_race to get data for proportions by binary_race and quarter relative to graduation
# then apply rounding and suppression
Figure_3_data <- Figure_2_data %>%
    # ignore extraneous columns
    select(-starts_with("fuzzy")) %>%
    inner_join(total_race, by = 'binary_race') %>%
    mutate(
        n_individuals_rounded = ifelse(individuals > 999, round(individuals, digits = -2), round(individuals, digits = -1)),
        n_total_rounded = ifelse(n_total > 999, round(n_total, digits = -2), round(n_total, digits = -1)),
        prop_rounded = ifelse(individuals < 11 | employers < 3, NA, round(n_individuals_rounded/n_total_rounded, digits = 2))
    )
Figure_3_data

We have calculated the proportion of our original cohort that has a wage. For example, in the first quarter after completing an apprenticeship, REDACTED of Non-White individuals from our original cohort are employed.

Notice that there is also at least a difference of 11 individuals between the true count by our binary race variable and quarter relative to apprenticeship completion and the total amount of individuals by binary race indicator that completed an apprenticeship, so no further redaction is required.

Let's save this data frame as a csv, as it will be required for the export submission process.

In [ ]:
write_csv(Figure_3_data, "Data/Figure_3_data.csv")

## Visualize the Data: Line Plot

We can create a line plot with `geom_line()`. Here, to dictate the color, we will use the `color` attribute as opposed to `fill`.

In [ ]:
Figure_3 <- Figure_3_data %>%
    ggplot(aes(x = Quarters_Relative_to_Completion, y = prop_rounded, color = binary_race)) +
    geom_line(size = 1.3) + 
    geom_point(size = 5) + 
    scale_color_manual("Binary Race Variable", values = binary_value_color) +
    labs(
        # Labelling x axis
        x = 'Quarter Relative to Completion', 
        # Labelling y axis
        y = 'Proportion Employed', 
        # Add a title that conveys the main takeaway of the graph
        title = 'REDACTED', 
        # cite the source of your data
        caption = 'Source: RAPIDS and UI Wages data'
    )

Figure_3

As currently constructed, the plot is a bit misleading, as the auto-generated limits on the y-axis make it appear as though non-white people were never employed after completing their apprenticeship. In addition to modifying the font sizes, we will update the range of the y-axis.

In [ ]:
Figure_3 <- Figure_3 + 
   theme(
        legend.text = element_text(size = 24), # legend text font size
        legend.title = element_text(size = 24), # legend title font size
        axis.text.x = element_text(size = 24), # x axis label font size
        axis.title.x = element_text(size = 24), # x axis title font size
        axis.text.y = element_text(size = 24), # y axis label font size
        axis.title.y = element_text(size = 24) # y axis title font size
    ) +
    ylim(0, 1) # update range of y-axis 

# Display the graph that we just created
Figure_3

# Employment Patterns

The final visualization in this notebook is a heatmap displaying our cohort of apprenticeship completers' employment patterns by quarter relative to completion, as we will focus on the 5 most common patterns. We do not use a heatmap in the classic way where each "box" in the map corresponds to a proportion or number. Instead, we will use the heatmap as a format to display employment patterns, as we will colorcode each box depending on if the pattern has or does not have employment in a specific quarter. 

## Steps for Export
To export a figure depicting unique patterns of employment, we will need to take the following steps:
- Find the number of unique employers and individuals that are associated with each employment pattern
- Select only those patterns that are associated with at least 11 invididuals and 3 employers
- Round the counts of individuals and population (`pop`)
- Create a percent variable using the rounded counts of individuals and population (`pop`)
- Round the percent
- Make the visual

First, though, we will need to create our underlying analytical frame.

## Prepare the Data for Export

We already have our linked cohort-wages data frame, where each row is a separate person/quarter combination. However, this data frame (**cohort_wages**) only contains entries of employment. First, we will get the information of the rest of the cohort by joining this data frame to our original **cohort** data frame.

In [ ]:
# get wage information for everyone in cohort
# if they don't have any wage information, they will have NA for all of the cohort_wages-specific variables
all_cohort_wages <- cohort %>%
    left_join(cohort_wages, by = c('apprnumber', 'binary_race', 'race'))

As a check, there should only be five possible values for the variable **Quarters_Relative_to_Completion**: 1, 2, 3, 4, and NA (if they were not in the wage records at all during this time frame).

In [ ]:
# confirm potential values of Quarters_Relative_to_Completion
all_cohort_wages %>% 
    distinct(Quarters_Relative_to_Completion)

For these individuals who did not match to **cohort_wages**, we will set the **Quarters_Relative_to_Completion** to 1, so that we will eventually be able to have one observation for each individual/quarter combination.

In [ ]:
all_cohort_wages <- all_cohort_wages %>%
    mutate(
        Quarters_Relative_to_Completion = ifelse(is.na(Quarters_Relative_to_Completion), 1, Quarters_Relative_to_Completion)
    )

# confirm potential values of Quarters_Relative_to_Completion
all_cohort_wages %>% 
    distinct(Quarters_Relative_to_Completion)

Now that we have all individuals, as well as instances of all desired `Quarters_Relative_to_Completion` values, we can leverage the tidyverse's `complete()` function, which will add additional rows for any person/quarter combinations that do not currently exist. 

> Note: If the person/quarter combination did not appear in the wage records, we will set their **Primary_Employer_Wages** value to NA.

In [ ]:
# complete file
completed <- all_cohort_wages %>%
    complete(apprnumber, Quarters_Relative_to_Completion, fill=list(Primary_Employer_Wages=NA))

# see that n should be a multiple of n_dist
completed %>%
    summarize(
        n = n(),
        n_inds = n_distinct(apprnumber),
        test = n_inds*4 == n
    )

Now that we have created **completed**, we just need to aggregate and manipulate the data frame so that each column is a quarter, and each observation is an individual, with the corresponding columns indicating whether the individual was employed in the given quarter. To start, let's create a variable `wage_ind`, which will be "yes" if the individual had greater than 0 earnings in the quarter, and "no" otherwise. Additionally, for each in column manipulation, we will change each `quarter_number` value from 1, 2, 3, 4 to Q1, Q2, Q3, Q4 and call this variable `quarter`.

In [ ]:
# create wage_ind and quarter variables
patterns <- completed %>%
    mutate(
        wage_ind = ifelse(Primary_Employer_Wages <= 0 | is.na(Primary_Employer_Wages), "no", "yes"),
        quarter = paste("Q", Quarters_Relative_to_Completion, sep="")
    )

head(patterns)

Now, we need to figure out how to "pivot" the data frame so that each column is a value of **quarter**, with **wage_ind** values for the **apprnumber** values. To do so, we will use `pivot_wider()`, which allows us to take a tidy data frame (one observation per row) and "widen" it so that each column becomes values from what was previously a single column (**quarter**) and the rows are occupied by those from a corresponding column (**wage_ind**). 

In [ ]:
# find most common employment patterns
patterns_wide <- patterns %>%
    select(apprnumber, quarter, wage_ind) %>%
    pivot_wider(names_from = quarter, values_from = wage_ind) %>%
    # after pivtor can summarize by each quarter column to find amount of people per row
    group_by(Q1, Q2, Q3, Q4) %>%
    summarize(
        ind_cnt = n_distinct(apprnumber)
    ) %>%
    arrange(desc(ind_cnt)) %>%
    ungroup()

head(patterns_wide)

Although we have the true number of individuals per pattern, we do not have any information about the number of employers. We can find the number of employers by taking the first three lines of the previous cell, which find the pattern for each individual:

    patterns %>%
        select(apprnumber, quarter, wage_ind) %>%
        pivot_wider(names_from = quarter, values_from = wage_ind)
        
And join that back to **cohort_wages** to get the employer information.

In [ ]:
emp_info <- patterns %>%
    select(apprnumber, quarter, wage_ind) %>%
    pivot_wider(names_from = quarter, values_from = wage_ind) %>% 
    left_join(cohort_wages, by = 'apprnumber') %>% 
    # find number of aggregate primary employers per grouping
    group_by(Q1, Q2, Q3, Q4) %>%
    summarize(
        emp_cnt = n_distinct(Federal_EIN, na.rm=T) #setting na.rm=T so we don't count NA employers
    )

head(emp_info)

In [ ]:
# top patterns with employer info
patterns_wide %>% 
    inner_join(emp_info, by = c("Q1", "Q2", "Q3", "Q4")) %>%
    head()

Before we can visualize this information, we need to apply the appropriate rounding and suppression rules.

> Note: **emp_cnt** technically only tracks primary employers - if there were a case where a pattern would be getting suppressed based on the employer count and not the individual count, that would warrant a separate query to load information on all employers in a quarter.

In [ ]:
# apply rounding and suppression
# ignore all redacted patterns
Figure_4_data <- patterns_wide %>% 
    inner_join(emp_info, by = c("Q1", "Q2", "Q3", "Q4")) %>%
    mutate(
        n_ind_cnt_rounded = ifelse(ind_cnt > 999, round(ind_cnt, digits = -2), round(ind_cnt, digits = -1)),
        n_total = sum(ind_cnt),
        n_total_rounded = ifelse(n_total > 999, round(n_total, digits = -2), round(n_total, digits = -1)),
        prop_rounded = ifelse(ind_cnt < 11 | (emp_cnt < 3 & emp_cnt > 0), NA, round(n_ind_cnt_rounded/n_total_rounded, digits = 2)), # don't want to redact 0 emp_cnt for the no (x4) pattern
        percent_rounded = percent(prop_rounded, digits = .01)
    ) %>%
    filter(!is.na(prop_rounded))

Figure_4_data

Now that the table has been prepared, let's save it as a csv.

In [ ]:
write_csv(Figure_4_data, "Data/Figure_4_data.csv")

## Visualize the Data: Heat map

We are going to convert the data back into a long format using the `pivot_longer()` before we can use the data as an input to `geom_tile()`, the function for creating heatmaps in `ggplot2`. Before doing so, let's save a vector of the rounded counts and percentages for each pattern for future reference in the visualization.

In [ ]:
# Save counts to use later in the heatmap - we cannot use the counts or percents as an index, as there could be duplicate values 
counts_percent_rounded <- Figure_4_data %>%
    mutate(
        counts_pcts = paste(Figure_4_data$n_ind_cnt_rounded, "(", Figure_4_data$percent_rounded,")")
    ) %>%
    pull(counts_pcts)

counts_percent_rounded

In [ ]:
# create data frame so that each row corresponds to a pattern/quarter/employment status observation
# seq_along() will create a vector to track each pattern
Figure_4_data_long <- Figure_4_data %>%
    mutate(
        Pattern = seq_along(1:nrow(Figure_4_data))
    ) %>%
    select(starts_with("Q"), Pattern) %>% 
    pivot_longer(names_to = 'Quarter', values_to = 'Employed', -Pattern) 

head(Figure_4_data_long)

Now we are ready to create the visualization using the `geom_tile()` layer:

In [ ]:
# Full code for the plot

levels = ordered(1:4)  # specify in which order to add the rows from our wide table (called "patterns") 
                                        

Figure_4_data_long$Quarter <- factor(Figure_4_data_long$Quarter, levels=c("Q1", "Q2", "Q3", "Q4")) # we want to preserve the same ordering of rows as they are sorted in the visual from first to last

Figure_4 <- Figure_4_data_long %>%
    ggplot(aes(x = Quarter, y = ordered(Pattern, levels=rev(levels)))) +                           # sort y-axis according to levels specified above
    geom_tile(aes(fill = Employed), colour = 'black') +                                            # fill the table with value from Employed column, create black contouring
    scale_fill_brewer("Employed", palette = "Paired") +                                            # specify a color palette
    scale_x_discrete(position = 'top') +                                                           # include x-axis labels on top of the plot
    labs(
        # Label Y axis
        y = "Counts (Percentages)",
        # Label X axis
        x = "Quarter After Completion" ,
        # Add a title that reflects the main takeaway of the figure
        title = "REDACTED",
        # Cite the source of your data
        caption = "Source: RAPIDS data and Arkansas UI Wage Records"
    ) +
    scale_y_discrete(labels=rev(counts_percent_rounded))  # rename the y-axis ticks to correspond to the counts from the table

Figure_4

In [ ]:
Figure_4 <- Figure_4 + theme(
        legend.text = element_text(size = 24), # legend text font size
        legend.title = element_text(size = 24), # legend title font size
        axis.text.x = element_text(size = 24), # x axis label font size
        axis.title.x = element_text(size = 24), # x axis title font size
        axis.text.y = element_text(size = 24), # y axis label font size
        axis.title.y = element_text(size = 24) # y axis title font size
    )

# Display the graph that we just created
Figure_4

# **Saving Visuals**

In this section, we look at the best ways to export our presentation-ready plots. We use `ggsave()` to save our visuals in a png, jpeg and pdf format without losing quality. In the following example, we only provide code for saving the first figure, but you would want to save each plot that you create for your project. 

## PNG

First, we provide an example of using ``ggsave`` with two parameters: `filename` and `plot`.

> **Note: In order to save the data in this way, you will need a folder called "Figures" in the same folder that contains your code/notebooks.**

In [ ]:
ggsave(
    filename = sprintf("Figures\\Figure_1.png"), # saving path
    plot = Figure_1 # plot name
)

This might not be the preferred way of saving a plot since the dimensions of the plot default to 6.67 x 6.67. We suggest looking at the file we just saved in its respective path. You will see how all the labels are cluttered and the graph can not be interpretted. Thus, we recommend using the `width` and `height` parameters in addition to `filename` and `plot`.

In [ ]:
ggsave(
    filename = sprintf("Figures\\Figure_1.png"), # saving path
    plot = Figure_2, # plot name
    width = 20, # width
    height = 12 # height
)

The code above saves plots in a format that can be interpretted conveniently. We can reuse this code to save in a JPEG and PDF format below:

## JPEG

In [ ]:
ggsave(
    filename = sprintf("Figures\\Figure_1.jpeg"), # saving path
    plot = Figure_3, # plot name
    width = 20, # width
    height = 12 # height
)

## PDF

In [ ]:
ggsave(
    filename = sprintf("Figures\\Figure_4.pdf"), # saving path
    plot = Figure_4, # plot name
    width = 20, # width
    height = 12 # height
)

# References
- Presentation Prep: Advanced, Applied Data Analytics Training, TANF Data Collabarative, 2022
- Presentation Prep: Beginner, Applied Data Analytics Training, TANF Data Collabarative, 2022
- Prince, Heath, Mian, Rukhshan, Feder, Benjamin, & Barrett, Nathan. (2022, April 4). Data Exploration and Linkage using Texas Unemployment Insurance Data. Zenodo. https://doi.org/10.5281/zenodo.6412649